In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dataset.zip 압축 풀기
!unzip --qq /content/drive/MyDrive/spectrograms.zip -d spectrograms

In [ ]:
# dataset.zip 압축 풀기
!unzip --qq /content/drive/MyDrive/ptb-xl.zip -d dataset

In [ ]:
pip install wfdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 11.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch.nn.functional as F
import warnings

In [ ]:
# 경고 메시지 숨기기
warnings.filterwarnings(action='ignore')

In [ ]:
# Dataset Class
class SpectrogramDataset(Dataset):
    def __init__(self, root_folder, img_size=(128, 128)):
        """
        Initialize the dataset by loading spectrogram images from folders.
        Args:
            root_folder (str): Root directory containing spectrograms organized by class.
            img_size (tuple): Size to resize images to (height, width).
        """
        self.data = []
        self.labels = []
        self.classes = []

        # Load images and labels
        for class_idx, class_name in enumerate(os.listdir(root_folder)):
            class_path = os.path.join(root_folder, class_name)
            if os.path.isdir(class_path):
                self.classes.append(class_name)
                for filename in os.listdir(class_path):
                    if filename.endswith((".png", ".jpg", ".jpeg")):
                        file_path = os.path.join(class_path, filename)
                        self.data.append(file_path)
                        self.labels.append(class_idx)

        self.img_size = img_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Load image
        img_path = self.data[idx]
        image = Image.open(img_path).convert("L")  # Grayscale
        image = image.resize(self.img_size)  # Resize to target size
        image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        # Load label
        label = self.labels[idx]
        return image, label

In [ ]:
# CNN2D Model
class CNN2D(nn.Module):
    def __init__(self, num_classes):
        super(CNN2D, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=100, kernel_size=(10, 10))
        self.conv2 = nn.Conv2d(in_channels=100, out_channels=250, kernel_size=(10, 10))
        self.conv3 = nn.Conv2d(in_channels=250, out_channels=500, kernel_size=(10, 10))
        self.conv4 = nn.Conv2d(in_channels=500, out_channels=1000, kernel_size=(10, 10))
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm2d(100)
        self.bn2 = nn.BatchNorm2d(250)
        self.bn3 = nn.BatchNorm2d(500)
        self.bn4 = nn.BatchNorm2d(1000)

        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout1 = nn.Dropout(0.5)
        self.fc = nn.Linear(1000, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout1(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x


In [ ]:
def main(spectrograms_dir):
    # 스펙트로그램 이미지와 슈퍼클래스 레이블 로드
    images, labels = load_spectrograms_with_superclass(spectrograms_dir)

    # 데이터셋 분할
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        images, labels, test_size=0.1, random_state=42, stratify=labels
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.2222, random_state=42, stratify=y_train_val
    )

    print(f"Total images: {len(images)}")
    print(f"Train set: {len(X_train)}, Validation set: {len(X_val)}, Test set: {len(X_test)}")
    print(f"Class distribution:")
    print(f"0 (Normal): {sum(1 for label in labels if label == 0)}")
    print(f"1 (Hyperdisease): {sum(1 for label in labels if label == 1)}")

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # 흑백 변환
        transforms.Resize((128, 128)),                # 고정된 크기로 변환
        transforms.ToTensor()
    ])


    # 데이터셋 생성
    train_dataset = SpectrogramDataset(X_train, y_train, transform)
    val_dataset = SpectrogramDataset(X_val, y_val, transform)
    test_dataset = SpectrogramDataset(X_test, y_test, transform)

    # 데이터 로더 생성
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    # 모델 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # ResNet 구성: [2, 2, 2, 2]는 ResNet18의 레이어 구성
    model = CNN2D_Binary()
    model = model.to(device)

    # 손실 함수와 옵티마이저
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 모델 훈련
    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer)

    # 테스트 세트 평가
    trained_model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

       # 혼동 행렬을 위한 변수
    confusion_matrix = torch.zeros(2, 2, dtype=torch.int64)

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = trained_model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            predicted = (outputs > 0.5).float()  # 확률 > 0.5를 기준으로 클래스 결정

            # 혼동 행렬 업데이트
            for t, p in zip(labels, predicted):
                confusion_matrix[t.long(), p.long()] += 1

            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = 100 * test_correct / test_total
    print(f'\nTest Accuracy: {test_accuracy:.2f}%')
    print('\n혼동 행렬:')
    print(confusion_matrix)

    # 정밀도, 재현율, F1 점수 계산
    tn, fp, fn, tp = confusion_matrix.numpy().ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f'\n정밀도: {precision:.4f}')
    print(f'재현율: {recall:.4f}')
    print(f'F1 점수: {f1_score:.4f}')

    # 혼동 행렬 시각화
    plt.figure(figsize=(8, 6))
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks([0, 1], ['Normal', 'Hyperdisease'])
    plt.yticks([0, 1], ['Normal', 'Hyperdisease'])

    # 혼동 행렬 값 추가
    for i in range(2):
        for j in range(2):
            plt.text(j, i, str(confusion_matrix[i, j].item()),
                     horizontalalignment="center",
                     color="white" if confusion_matrix[i, j] > confusion_matrix.max()/2 else "black")

    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

In [ ]:
if __name__ == '__main__':
    spectrograms_dir = '/content/spectrograms'
    main(spectrograms_dir)

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.